# Case Studies
# Case 1: Caret 6 (SXSW)
![Parametric vault rendering 1](test-repo/images/Caret_6_Photo_by_Casey_Dunn_03.jpg)

central catenary vault + cantilevering arms + cascading vault

This script was developed with assistance from ChatGPT, which helped translate the geometric and topological design logic into parametric Python code.

In [ ]:
# 1. Vault Base Geometry 參數化（Typology）

def catenary(x, span=10, height=4):
    """
    生成懸鏈線 (catenary curve)
    """
    a = height
    return a * np.cosh((x - span/2)/a) - a


def generate_vault(span=10, height=4, divisions=40, layers=6, twist=0.3):
    """
    生成多層 vault（SXSW 類型）：有 twist + 多層高程差
    """
    all_ribs = []

    x = np.linspace(0, span, divisions)

    for i in range(layers):
        h = height * (1 + 0.10 * sin(i * 0.8))          # 每層不同高度
        z_offset = i * 0.4                              # 層間高度
        angle = twist * i                               # 螺旋扭轉量

        pts = []
        for xi in x:
            y = catenary(xi, span, h)
            X = xi * cos(angle) - y * sin(angle)
            Y = xi * sin(angle) + y * cos(angle)
            Z = z_offset
            pts.append(Point(X, Y, Z))

        rib = Polyline(pts)
        all_ribs.append(rib)

    return all_ribs

# 2. Topological Logic（生成 Rib 網絡）

def connect_ribs(ribs):
    """
    連結 rib 之間的橫向線材（形成 vault 網絡）
    """
    network = []

    for i in range(len(ribs) - 1):
        ribA = ribs[i].points
        ribB = ribs[i + 1].points

        for pa, pb in zip(ribA, ribB):
            network.append(Polyline([pa, pb]))

    return network

# 3. Fabrication Panels（將 vault 轉成可製造單元）

def ribs_to_panels(ribs, thickness=0.05):
    """
    將 rib 轉成面（簡化版：兩條線之間做薄板）
    """
    panels = []

    for rib in ribs:
        pts = rib.points
        for i in range(len(pts) - 1):
            p1 = pts[i]
            p2 = pts[i + 1]

            # 偏移方向（簡化：往 z 軸偏移）
            offset = Point(p1.x, p1.y, p1.z + thickness)
            offset2 = Point(p2.x, p2.y, p2.z + thickness)

            panel = Polyline([p1, p2, offset2, offset, p1])
            panels.append(panel)

    return panels

# MAIN

ribs = generate_vault(
    span=12,
    height=4,
    divisions=40,
    layers=8,
    twist=0.25
)

network = connect_ribs(ribs)
panels = ribs_to_panels(ribs)

# Viewer 顯示

viewer = Viewer()

for rib in ribs:
    viewer.scene.add(rib, linewidth=3, color=(0.2, 0.7, 1))

for e in network:
    viewer.scene.add(e, linewidth=1, color=(1, 0.8, 0.2))

for p in panels:
    viewer.scene.add(p, facecolor=(0.8, 0.8, 0.8), opacity=0.6)

viewer.show()